## PDF2Markdown

- pymupdf4llm

In [2]:
# import pymupdf4llm
# FILE_PATH = "/home/taejong_kim/workspace/rag-lab/database/BA_사용메뉴얼_분석가.pdf"
# IMG_CONTENT_PATH = "/home/taejong_kim/workspace/rag-lab/database/image"

# md_text = pymupdf4llm.to_markdown(
#     FILE_PATH, write_images=True, image_path=IMG_CONTENT_PATH)

In [3]:
# import re

# splited_markdonw = [item for item in re.split(
#     r'(?<!##)##(?!##)', md_text) if item != '']

In [4]:
# from langchain.schema import Document


# def extract_images_and_create_documents(text):
#     """
#     Markdown 텍스트에서 ![]() 형식의 이미지를 추출하고, 
#     본문과 이미지 리스트를 LangChain Document 객체로 변환
#     """
#     # 이미지 패턴 탐색 (예: ![alt text](image_url))
#     image_pattern = re.compile(r'!\[.*?\]\((.*?)\)')

#     # 이미지 리스트 추출
#     images = image_pattern.findall(text)

#     # 본문에서 이미지 태그 제거
#     cleaned_text = image_pattern.sub('', text).strip()

#     # LangChain Document 생성
#     return Document(page_content=cleaned_text, metadata={"images": images})


# docs4md = []
# for md in splited_markdonw:
#     doc = extract_images_and_create_documents(md)
#     docs4md.append(doc)

## Vector Store


In [5]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import os
from dotenv import load_dotenv
import pickle

load_dotenv()

True

### HuggingFace
- model = 'jhgan/ko-sroberta-multitask'

In [ ]:
# hf = HuggingFaceEmbeddings(
#     model_name='jhgan/ko-sroberta-multitask')  # embedding model

# vectorstore = FAISS.from_documents(docs4md, embedding=hf)  # vector store
# retriever = vectorstore.retriever()

In [9]:
VECTORSTORE_PATH = "/home/taejong_kim/workspace/rag-lab/database/vector_store"
EMBEDDING_MODEL_PATH = "/home/taejong_kim/workspace/rag-lab/database/embedding_model" + \
    "/hf_embedding_model01.pkl"

# vectorstore.save_local(VECTORSTORE_PATH)

# with open(EMBEDDING_MODEL_PATH, "wb") as f:
#     pickle.dump(hf, f)

In [10]:
# 1. 저장된 임베딩 모델 불러오기

with open(EMBEDDING_MODEL_PATH, "rb") as f:
    embedding_model = pickle.load(f)

# 2. 저장된 벡터스토어 불러오기
vectorstore = FAISS.load_local(
    VECTORSTORE_PATH, embedding_model, allow_dangerous_deserialization=True)


# 3. 검색
query = "분석가"
results = vectorstore.similarity_search(query, top_k=5)

print('검색 결과: ', results[0].page_content)

검색 결과:  목록

1. BRIQUE Analytics에서 다중처리의 의미

2. 알고리즘 작성

3. 분석플로우 작성 및 실행


In [ ]:
image_paths = [doc.metadata['images'] for doc in results]
image_paths

# results[1].metadata['images']

[[], [], [], []]

In [15]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever()

itemgetter(query) | retriever | StrOutputParser()

RunnableLambda(itemgetter('분석가'))
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f987582fbf0>, search_kwargs={})
| StrOutputParser()

In [43]:
for doc in retriever.get_relevant_documents(query, top_k=5):
    img_path = doc.metadata["images"]
    if len(img_path) > 0:
        for img in img_path:
            print(img)

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


prompt = PromptTemplate.from_template(
    """"You are a support agent. 
Please respond in the same language as the user's input. Detect the language they are using and reply naturally in that language while maintaining clarity and accuracy.

If you don't know the answer, just say that you don't know. 

Use the following pieces of retrieved context to answer the question. 

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 언어 모델
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | llm
    # | JsonOutputParser()
)


In [29]:
query = {'question':'워크플로우 생성 방법'}
answer = chain.invoke(query)

In [31]:
answer.model_dump()

{'content': '제공된 문서에는 워크플로우 생성 방법에 대한 자세한 설명은 나와 있지 않습니다. 워크플로우를 "여는" 방법이나 워크플로우의 "실행 엔진 정보 조회" 방법에 대한 내용은 있지만,  "생성" 방법에 대한 구체적인 내용은 없습니다.  더 자세한 정보를 원하시면 다른 문서를 참고하시거나, 워크플로우 관련 매뉴얼을 확인해 보시기 바랍니다.',
 'additional_kwargs': {},
 'response_metadata': {'prompt_feedback': {'block_reason': 0,
   'safety_ratings': []},
  'finish_reason': 'STOP',
  'safety_ratings': []},
 'type': 'ai',
 'name': None,
 'id': 'run-875dcbf5-8997-4b40-b40f-1b0cd99e33a8-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 772,
  'output_tokens': 118,
  'total_tokens': 890,
  'input_token_details': {'cache_read': 0}}}

In [37]:
query['question']

'워크플로우 생성 방법'

In [36]:
img_list = []

for doc in retriever.invoke(query['question']):
    img_path = doc.metadata["images"]
    if len(img_path) > 0:
        for img in img_path:
            img_list.append(img)
            
img_list

['/home/taejong_kim/workspace/rag-lab/database/image/BA_사용메뉴얼_분석가.pdf-172-0.png']